# Deploying Machine Learning Models using ksonnet and Ambassador
## Experimental


## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - [Minikube](https://github.com/kubernetes/minikube) version v0.24.0 or greater
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)
 - [ksonnet client](https://ksonnet.io/)

Start minikube and ensure custom resource validation is activated and ther is 5G of memory.

In [ ]:
!minikube start --memory=5000 --feature-gates=CustomResourceValidation=true

## Create Namespace

In [ ]:
!kubectl create namespace seldon

## Install Ambassador
See the Abassador [getting started](https://www.getambassador.io/user-guide/getting-started) docs. Eventually, this would also be done via ksonnet.

In [ ]:
!kubectl apply -f resources/ambassador-service.yaml -n seldon

In [ ]:
!kubectl apply -f https://getambassador.io/yaml/ambassador/ambassador-no-rbac.yaml -n seldon

## Install Seldon Core
Create a ksonnet app and install the prototypes from our registry.

In [ ]:
!ks init my-ml-deployment --api-spec=version:v1.8.0

In [ ]:
!cd my-ml-deployment && \
    ks registry add seldon-core github.com/SeldonIO/seldon-core/tree/master/seldon-core && \
    ks pkg install seldon-core/seldon-core@master && \
    ks generate seldon-core seldon-core --withApife=false --namespace=seldon

In [ ]:
!cd my-ml-deployment && \
      ks apply default

## Set up REST and gRPC methods

Install gRPC modules for the prediction protos.

In [ ]:
!cp ../proto/prediction.proto ./proto
!python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto

Illustration of both REST and gRPC requests. 

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
from proto import prediction_pb2
from proto import prediction_pb2_grpc
try:
    from commands import getoutput # python 2
except ImportError:
    from subprocess import getoutput # python 3

NAMESPACE='seldon'
MINIKUBE_IP=getoutput('minikube ip')
AMBASSADOR_PORT=getoutput("kubectl get svc -n "+NAMESPACE+" -l service=ambassador -o jsonpath='{.items[0].spec.ports[0].nodePort}'")

def rest_request(deploymentName):
    payload = {"data":{"names":["a","b"],"tensor":{"shape":[2,2],"values":[0,0,1,1]}}}
    response = requests.post(
                "http://"+MINIKUBE_IP+":"+AMBASSADOR_PORT+"/seldon/"+deploymentName+"/api/v0.1/predictions",
                json=payload)
    print(response.status_code)
    print(response.text)    
    
def rest_request_auth(deploymentName,username,password):
    payload = {"data":{"names":["a","b"],"tensor":{"shape":[2,2],"values":[0,0,1,1]}}}
    response = requests.post(
                "http://"+MINIKUBE_IP+":"+AMBASSADOR_PORT+"/seldon/"+deploymentName+"/api/v0.1/predictions",
                json=payload,
                auth=HTTPBasicAuth(username, password))
    print(response.status_code)
    print(response.text)

def grpc_request(deploymentName):
    datadef = prediction_pb2.DefaultData(
            names = ["a","b"],
            tensor = prediction_pb2.Tensor(
                shape = [3,2],
                values = [1.0,1.0,2.0,3.0,4.0,5.0]
                )
            )
    request = prediction_pb2.SeldonMessage(data = datadef)
    channel = grpc.insecure_channel(MINIKUBE_IP+":"+AMBASSADOR_PORT)
    stub = prediction_pb2_grpc.SeldonStub(channel)
    metadata = [('seldon',deploymentName)]
    response = stub.Predict(request=request,metadata=metadata)
    print(response)

## Create Seldon Deployment

**Check everything is running before continuing**

In [ ]:
!kubectl get pods -n seldon

In [ ]:
!kubectl apply -f resources/model.json -n seldon

Check status of deployment before continuing. **ReplicasAvailable must be equal to 1**  First time might take some time to download images.

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}' -n seldon

## Get predictions

#### REST Request

In [ ]:
rest_request("seldon-deployment-example")

#### gRPC Request

In [ ]:
grpc_request("seldon-deployment-example")

## Adding Authentication
We will add the example authentication from the Ambassador tutorial.

In [ ]:
!kubectl apply -f resources/ambassador-auth-service-setup.yaml -n seldon

** Need to wait until running before adding Ambassador config **

In [ ]:
!kubectl get pods -n seldon

In [ ]:
!kubectl apply -f resources/ambassador-auth-service-config.yaml -n seldon

Show failed request when auth is running

In [ ]:
rest_request("seldon-deployment-example")

Show successful request with auth

In [ ]:
rest_request_auth("seldon-deployment-example","username","password")

# Tear down

In [ ]:
!kubectl delete -f resources/ambassador-auth-service-setup.yaml -n seldon

In [ ]:
!kubectl delete -f https://getambassador.io/yaml/ambassador/ambassador-no-rbac.yaml -n seldon

In [ ]:
!kubectl delete -f resources/model.json

In [ ]:
!cd my-ml-deployment && ks delete default

In [ ]:
!rm -rf my-ml-deployment